In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

from ecephys.data import paths
from ecephys.utils.utils import load_df_h5
from ecephys.scoring import load_visbrain_hypnogram

In [6]:
pd.options.mode.chained_assignment = None  # default='warn'

## Read the data

In [7]:
SUBJECT = "Doppio"
BSL = "BSL-0+2"
REC = "REC-0+2"

In [8]:
bl_hypno = load_visbrain_hypnogram(paths.get_datapath(subject=SUBJECT, condition=BSL, data="hypnogram.txt"))
with pd.HDFStore(paths.get_datapath(subject=SUBJECT, condition=BSL, data="sharp_waves.h5")) as store:
    spws_bl, metadata_bl = load_df_h5(store)

In [9]:
spws_bl["sink_amplitude"] = -spws_bl["sink_amplitude"]
spws_bl["sink_integral"] = -spws_bl["sink_integral"]

In [10]:
rec_hypno = load_visbrain_hypnogram(paths.get_datapath(subject=SUBJECT, condition=REC, data="hypnogram.txt"))
with pd.HDFStore(paths.get_datapath(subject=SUBJECT, condition=REC, data="sharp_waves.h5")) as store:
    spws_rec, metadata_rec = load_df_h5(store)

In [11]:
spws_rec["sink_amplitude"] = -spws_rec["sink_amplitude"]
spws_rec["sink_integral"] = -spws_rec["sink_integral"]

## Look at basic summary statistics

In [12]:
bl_total_NREM = bl_hypno.loc[bl_hypno['state'] == 'N2'].duration.sum()
bl_total_NREM

3407.9599609375

In [13]:
rec_total_NREM = rec_hypno.loc[rec_hypno['state'] == 'N2'].duration.sum()
rec_total_NREM

4388.0

In [90]:
len(spws_bl)

3720

In [91]:
len(spws_rec)

4760

In [16]:
spws_bl.median()

start_time        3388.735592
end_time          3388.817592
duration             0.056800
midpoint          3388.776592
sink_amplitude       0.041947
sink_integral        1.575076
dtype: float64

In [17]:
spws_rec.median()

start_time        3754.461425
end_time          3754.529824
duration             0.059200
midpoint          3754.495624
sink_amplitude       0.043342
sink_integral        1.696058
dtype: float64

In [18]:
spws_bl.mean()

start_time        3488.930588
end_time          3488.995686
duration             0.065098
midpoint          3488.963137
sink_amplitude       0.045456
sink_integral        1.913661
dtype: float64

In [19]:
spws_rec.mean()

start_time        3681.813292
end_time          3681.881774
duration             0.068482
midpoint          3681.847533
sink_amplitude       0.050170
sink_integral        2.249689
dtype: float64

## Look at feature distributions and correlations

In [20]:
ax = scatter_matrix(spws_bl[["duration", "sink_amplitude", "sink_integral"]], figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
ax2 = scatter_matrix(spws_bl[["duration", "sink_amplitude", "sink_integral"]].applymap(np.log), figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
ax = scatter_matrix(spws_rec[["duration", "sink_amplitude", "sink_integral"]], figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
ax2 = scatter_matrix(spws_rec[["duration", "sink_amplitude", "sink_integral"]].applymap(np.log), figsize=(10, 10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Look at log-normally distributed features

In [24]:
import scipy.stats as stats

def plot_qq(bl, rec, var):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    
    stats.probplot(bl[var], dist="norm", plot=axes[0, 0])
    axes[0, 0].set_title("Baseline {}".format(var))
    stats.probplot(np.log(bl[var]), dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title("Baseline log {}".format(var))
    stats.probplot(rec[var], dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title("Recovery {}".format(var))
    stats.probplot(np.log(rec[var]), dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title("Recovery log {}".format(var))

    plt.show()

In [25]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_acf_pacf(bl, rec, var, lags=None):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))

    plot_acf(bl[var], lags=lags, ax=axes[0, 0], title="Baseline {} acf".format(var))
    plot_pacf(bl[var], lags=lags, ax=axes[0, 1], title="Baseline {} pacf".format(var))
    plot_acf(rec[var], lags=lags, ax=axes[1, 0], title="Recovery {} acf".format(var))
    plot_pacf(rec[var], lags=lags, ax=axes[1, 1], title="Recovery {} pacf".format(var))

    plt.show()

In [26]:
def plot_vs_time(bl, rec, var):
    fig, axes = plt.subplots(2, 1)
    bl.plot.scatter(x='midpoint', y=var, ax=axes[0], title='Baseline', s=1)
    rec.plot.scatter(x='midpoint', y=var, ax=axes[1], title='Recovery', s=1)
    plt.show()

In [27]:
def test_slope(spws, x, y):
    return stats.linregress(spws[x], spws[y])

In [28]:
# https://dfrieds.com/math/effect-size.html
def interpret_cohens_d(cohens_d):
    """
    Determines text interpretation of effect size given Cohen's d value

    :param cohens_d: float of Cohen's d value
    :returns: effect_size_interpretation: adjective to describe magnitude of effect size
    """
    if 0<=cohens_d<0.1:
        effect_size_interpretation = "Very Small"
    elif 0.1<=cohens_d<0.35:
        effect_size_interpretation = "Small"
    elif 0.35<=cohens_d<0.65:
        effect_size_interpretation = "Medium"
    elif 0.65<=cohens_d<0.9:
        effect_size_interpretation = "Large"
    elif cohens_d >= 0.9:
        effect_size_interpretation = "Very Large"
    return effect_size_interpretation

In [29]:
def two_sample_data_setup_for_cohens_d(mean_1, mean_2, sample_std_dev_1, sample_std_dev_2):
    """
    Creates two normal distributions of data values. 
    Calculates differences in values and standard deviation of list of differences.
    Calculates Cohen's D - equation used in dependent t-tests

    :param mean_1: mean of group one
    :param mean_2: mean of group two
    :param sample_std_dev_1: sample standard deviation of group one
    :param sample_std_dev_2: sample standard deviation of group two
    
    :returns: values_1, values_2, cohens_d: values for group one, values for group two, Cohen's d
    """
    values_1 = np.random.normal(loc=mean_1, scale=sample_std_dev_1, size=3000)
    values_2 = np.random.normal(loc=mean_2, scale=sample_std_dev_2, size=3000)
    differences_group_values = np.subtract(values_1, values_2)
    std_dev_differences_group_values = np.std(differences_group_values, ddof=1)
    cohens_d = round(abs((mean_1 - mean_2)/std_dev_differences_group_values), 2)
    return values_1, values_2, cohens_d

In [30]:
import seaborn as sns

def visualize_distributions(values_1, values_2, cohens_d, cohens_d_interpretation, xlabel):
    """
    Plots KDE plots of values from values_1 and values_2.
    Inserts Cohen's d value and interpretation in title

    :param values_1: values in group one
    :param values_2: values in group two 
    :param cohens_d: Cohen's d value
    :param cohens_d_interpretation: text to describe magnitude of effect size
    :returns: plot figure
    """
    plt.figure(figsize=(10, 8))
    sns.distplot(values_1, hist=False, label='Baseline')
    sns.distplot(values_2, hist=False, label='Recovery')
    plt.xlabel(xlabel, labelpad=13)
    plt.ylabel("frequency", labelpad=13)
    plt.legend()
    plt.title("Distribution of Two Groups with {0} Cohen's d of {1}".format(cohens_d_interpretation, cohens_d), y=1.015, fontsize=22)

### Duration

In [31]:
plot_qq(spws_bl, spws_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plot_acf_pacf(spws_bl, spws_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plot_vs_time(spws_bl, spws_rec, 'duration')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
test_slope(spws_bl, 'midpoint', 'duration')

LinregressResult(slope=-3.7816307207918526e-07, intercept=0.06641761112537657, rvalue=-0.024363845253175424, pvalue=0.13735426670867776, stderr=2.5447765612414237e-07)

In [35]:
test_slope(spws_rec, 'midpoint', 'duration')

LinregressResult(slope=-3.440786046043307e-07, intercept=0.0697488769361647, rvalue=-0.01917157892119765, pvalue=0.18601030905181695, stderr=2.601403622720611e-07)

In [36]:
stats.ttest_ind(np.log(spws_bl.duration), np.log(spws_rec.duration), equal_var=False)

Ttest_indResult(statistic=-3.584550382279915, pvalue=0.0003396083668520128)

In [37]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(np.log(spws_bl.duration).mean(), 
                                                                     np.log(spws_rec.duration).mean(), 
                                                                     np.log(spws_bl.duration).std(), 
                                                                     np.log(spws_rec.duration).std())
visualize_distributions(np.log(spws_bl.duration), np.log(spws_rec).duration, cohens_d, interpret_cohens_d(cohens_d), xlabel="log transformed sink duration")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [38]:
stat, p, med, tbl = stats.median_test(spws_bl.duration, spws_rec.duration) 
print(stat, p, med)

10.063801711068749 0.0015121088100153676 0.057999941675006994


In [39]:
stats.kruskal(spws_bl.duration, spws_rec.duration)

KruskalResult(statistic=12.277326242495, pvalue=0.0004584947862548842)

In [40]:
stats.mannwhitneyu(spws_bl.duration, spws_rec.duration)

MannwhitneyuResult(statistic=8461625.0, pvalue=0.00022925124072369115)

### Sink integral

In [41]:
plot_qq(spws_bl, spws_rec, 'sink_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
plot_acf_pacf(spws_bl, spws_rec, 'sink_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
plot_vs_time(spws_bl, spws_rec, 'sink_integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
test_slope(spws_bl, 'midpoint', 'sink_integral')

LinregressResult(slope=-1.4911113209219455e-05, intercept=1.9656853893569282, rvalue=-0.02609855361642886, pvalue=0.11148946243201982, stderr=9.366794203990797e-06)

In [45]:
test_slope(spws_rec, 'midpoint', 'sink_integral')

LinregressResult(slope=-6.166101951847556e-05, intercept=2.4767153806429865, rvalue=-0.07447084582174938, pvalue=2.6931840463553804e-07, stderr=1.1970287068225474e-05)

In [46]:
stats.ttest_ind(np.log(spws_bl.sink_integral), np.log(spws_rec.sink_integral), equal_var=False)

Ttest_indResult(statistic=-7.869336676557132, pvalue=4.007192562681044e-15)

In [47]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(np.log(spws_bl.sink_integral).mean(), 
                                                                     np.log(spws_rec.sink_integral).mean(), 
                                                                     np.log(spws_bl.sink_integral).std(), 
                                                                     np.log(spws_rec.sink_integral).std())
visualize_distributions(np.log(spws_bl.sink_integral), np.log(spws_rec).sink_integral, cohens_d, interpret_cohens_d(cohens_d), xlabel="log-transformed sink integral")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [48]:
stat, p, med, tbl = stats.median_test(spws_bl.duration, spws_rec.duration) 
print(stat, p, med)

10.063801711068749 0.0015121088100153676 0.057999941675006994


In [49]:
stats.kruskal(spws_bl.duration, spws_rec.duration)

KruskalResult(statistic=12.277326242495, pvalue=0.0004584947862548842)

In [50]:
stats.mannwhitneyu(spws_bl.duration, spws_rec.duration)

MannwhitneyuResult(statistic=8461625.0, pvalue=0.00022925124072369115)

## Look at non-normally distributed features

### Sink amplitude

In [51]:
plot_acf_pacf(spws_bl, spws_rec, 'sink_amplitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plot_vs_time(spws_bl, spws_rec, 'sink_amplitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
test_slope(spws_bl, 'midpoint', 'sink_amplitude')

LinregressResult(slope=-1.1561550546279516e-07, intercept=0.045859688530369154, rvalue=-0.01869738919025395, pvalue=0.2542430280135278, stderr=1.0139218802594485e-07)

In [54]:
test_slope(spws_rec, 'midpoint', 'sink_amplitude')

LinregressResult(slope=-1.0444886540937511e-06, intercept=0.05401612938034222, rvalue=-0.11066462816655338, pvalue=1.9095173548786838e-14, stderr=1.3598998699206676e-07)

In [55]:
stats.mannwhitneyu(spws_bl.sink_amplitude, spws_rec.sink_amplitude)

MannwhitneyuResult(statistic=7958368.0, pvalue=6.094250460747972e-16)

In [56]:
plt.figure(figsize=(10, 8))
sns.distplot(spws_bl.sink_amplitude, hist=False, label='Baseline')
sns.distplot(spws_rec.sink_amplitude, hist=False, label='Recovery')
plt.xlabel("Sink ampliutde (mA/mm)", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


## Look at spw density

In [57]:
plt.figure(figsize=(10, 8))
sns.distplot(bl_hypno[bl_hypno.state == "N2"].duration, hist=False, label='Baseline')
sns.distplot(rec_hypno[rec_hypno.state == "N2"].duration, hist=False, label='Recovery')
plt.xlabel("NREM Episode Duration", labelpad=13)
plt.ylabel("frequency", labelpad=13)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [58]:
def get_spw_density(hypno, spws):  
    N2 = hypno[hypno.state == "N2"]
    N2['n_spws'] = N2.apply(lambda episode: np.sum(np.logical_and(spws.midpoint >= episode.start_time, spws.midpoint <= episode.end_time)), axis=1)
    N2['spw_density'] = N2.apply(lambda episode: episode.n_spws / episode.duration, axis=1)
    return N2

In [59]:
bl_density = get_spw_density(bl_hypno, spws_bl)

plt.figure()
ax = bl_density.spw_density.plot.hist(bins=20)

print("Mean spw density: ", bl_density.spw_density.mean(), "\n", "Median spw density: ", bl_density.spw_density.median())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  0.8644307769497434 
 Median spw density:  0.8836580086580086


In [60]:
ax = bl_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
test_slope(bl_density, 'start_time', 'spw_density')

LinregressResult(slope=4.812618661657016e-07, intercept=0.8627541188507413, rvalue=0.005225881787592856, pvalue=0.9700832329485374, stderr=1.2770687185421427e-05)

In [62]:
rec_density = get_spw_density(rec_hypno, spws_rec)

plt.figure()
ax = rec_density.spw_density.plot.hist(bins=20)

print("Mean spw density: ", rec_density.spw_density.mean(), "\n", "Median spw density: ", rec_density.spw_density.median())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  0.8436879131595025 
 Median spw density:  0.8958333333333334


In [63]:
plt.figure()
rec_density.boxplot(column=['spw_density'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [64]:
ax = rec_density.plot(x='start_time', y='spw_density')

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:328: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
test_slope(rec_density, 'start_time', 'spw_density')

LinregressResult(slope=-4.111699617710325e-06, intercept=0.857823020842782, rvalue=-0.037171452265867946, pvalue=0.8040893611662923, stderr=1.6478032311798006e-05)

In [66]:
plot_qq(bl_density, rec_density, 'spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
stats.ttest_ind(bl_density.spw_density, rec_density.spw_density, equal_var=False)

Ttest_indResult(statistic=0.47683010528282066, pvalue=0.634657427796079)

In [68]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(bl_density.spw_density.mean(), 
                                                                     rec_density.spw_density.mean(), 
                                                                     bl_density.spw_density.std(), 
                                                                     rec_density.spw_density.std())
visualize_distributions(bl_density.spw_density, rec_density.spw_density, cohens_d, interpret_cohens_d(cohens_d))


TypeError: visualize_distributions() missing 1 required positional argument: 'xlabel'

In [69]:
stat, p, med, tbl = stats.median_test(bl_density.spw_density, rec_density.spw_density) 
print(stat, p, med)

0.008618373275235937 0.9260344908326814 0.8863636363636364


In [70]:
stats.kruskal(bl_density.spw_density, rec_density.spw_density)

KruskalResult(statistic=0.025608673998291212, pvalue=0.8728597234874943)

In [71]:
stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density)

MannwhitneyuResult(statistic=1245.5, pvalue=0.43777127434845425)

## Epoched spw density

In [72]:
# https://stackoverflow.com/questions/21160134/flatten-a-column-with-value-of-type-list-while-duplicating-the-other-columns-va
def unnest_df(df, col, reset_index=False):
    import pandas as pd
    col_flat = pd.DataFrame([[i, x] 
                       for i, y in df[col].apply(list).iteritems() 
                           for x in y], columns=['I', col])
    col_flat = col_flat.set_index('I')
    df = df.drop(col, 1)
    df = df.merge(col_flat, left_index=True, right_index=True)
    if reset_index:
        df = df.reset_index(drop=True)
    return df

def _get_spw_density(hypno, spws, epoch_length):  
    N2 = hypno[np.logical_and(hypno.state == "N2", hypno.duration >= epoch_length)]
    N2['epoch_start_times'] = N2.apply(lambda episode: np.arange(episode.start_time, episode.end_time - epoch_length, epoch_length), axis=1)
    N2 = unnest_df(N2, 'epoch_start_times')
    N2 = N2.rename(columns={'end_time': 'episode_end_time', 'start_time': 'epsiode_start_time', 'duration': 'episode_duration', 'epoch_start_times': 'start_time'})
    N2['end_time'] = N2.apply(lambda epoch: epoch.start_time + epoch_length, axis=1)
    N2['n_spws'] = N2.apply(lambda epoch: np.sum(np.logical_and(spws.midpoint >= epoch.start_time, spws.midpoint <= epoch.end_time)), axis=1)
    N2['spw_density'] = N2.apply(lambda epoch: epoch.n_spws / epoch_length, axis=1)
    return N2

In [73]:
epoch_length = 10

In [74]:
bl_density = _get_spw_density(bl_hypno, spws_bl, epoch_length)

plt.figure()
ax = bl_density.spw_density.plot.hist()

print("Mean spw density: ", bl_density.spw_density.mean(), "\n", "Median spw density: ", bl_density.spw_density.median(), "\n", "Number of spws: ", len(bl_density))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  0.9 
 Median spw density:  0.9 
 Number of spws:  309


In [75]:
ax = bl_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
test_slope(bl_density, 'start_time', 'spw_density')

LinregressResult(slope=-1.6311885342415435e-05, intercept=0.9584730380392934, rvalue=-0.099659857289227, pvalue=0.08027074485072504, stderr=9.294953131637295e-06)

In [77]:
rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)

plt.figure()
ax = rec_density.spw_density.plot.hist()

print("Mean spw density: ", rec_density.spw_density.mean(), "\n", "Median spw density: ", rec_density.spw_density.median(), "\n", "Number of spws: ", len(rec_density))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean spw density:  1.0194103194103195 
 Median spw density:  1.0 
 Number of spws:  407


In [78]:
ax = rec_density.plot(x='start_time', y='spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
test_slope(rec_density, 'start_time', 'spw_density')

LinregressResult(slope=1.1395380709390015e-05, intercept=0.9764634540738925, rvalue=0.06034506402963296, pvalue=0.2244504326181378, stderr=9.366285342032842e-06)

In [80]:
plot_qq(bl_density, rec_density, 'spw_density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/lib/function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/numpy/lib/function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [81]:
stats.ttest_ind(bl_density.spw_density, rec_density.spw_density, equal_var=False)

Ttest_indResult(statistic=-4.451122072701914, pvalue=9.985580085856318e-06)

In [82]:
values_bl, values_rec, cohens_d = two_sample_data_setup_for_cohens_d(bl_density.spw_density.mean(), 
                                                                     rec_density.spw_density.mean(), 
                                                                     bl_density.spw_density.std(), 
                                                                     rec_density.spw_density.std())
visualize_distributions(bl_density.spw_density, rec_density.spw_density, cohens_d, interpret_cohens_d(cohens_d), xlabel="Event density (per sec)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)
/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/seaborn/distributions.py:2551: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [83]:
stat, p, med, tbl = stats.median_test(bl_density.spw_density, rec_density.spw_density) 
print(stat, p, med)

22.804022782811217 1.7938981954373195e-06 1.0


In [84]:
stats.kruskal(bl_density.spw_density, rec_density.spw_density)

KruskalResult(statistic=23.169063238455415, pvalue=1.483650654518776e-06)

In [85]:
stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density)

MannwhitneyuResult(statistic=49732.5, pvalue=7.425053677111444e-07)

### Sensitivity Analysis

In [86]:
def _get_density_ttest_pval(epoch_length):
    bl_density = _get_spw_density(bl_hypno, spws_bl, epoch_length)
    rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)
    return stats.ttest_ind(bl_density.spw_density, rec_density.spw_density, equal_var=False).pvalue

def _get_density_MannWhitney_pval(epoch_length):
    bl_density = _get_spw_density(bl_hypno, spws_bl, epoch_length)
    rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)
    return stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density).pvalue

def _get_density_nEpochs(hypno, spws, epoch_length):
    bl_density = _get_spw_density(hypno, spws_bl, epoch_length)
    rec_density = _get_spw_density(rec_hypno, spws_rec, epoch_length)
    return stats.mannwhitneyu(bl_density.spw_density, rec_density.spw_density).pvalue

df = pd.DataFrame(data={'epoch_length': np.arange(5, 65, 5)})
df['ttest_pval'] = df.apply(lambda x: _get_density_ttest_pval(x.epoch_length), axis=1)
df['mw_pval'] = df.apply(lambda x: _get_density_MannWhitney_pval(x.epoch_length), axis=1)
df['n_epochs_bl'] = df.apply(lambda x: len(_get_spw_density(bl_hypno, spws_bl, x.epoch_length)), axis=1)
df['n_epochs_rec'] = df.apply(lambda x: len(_get_spw_density(rec_hypno, spws_rec, x.epoch_length)), axis=1)

In [87]:
df

,epoch_length,ttest_pval,mw_pval,n_epochs_bl,n_epochs_rec
0,5,0.000015,2.593030e-06,651,845
1,10,0.000010,7.425054e-07,309,407
2,15,0.000011,3.106156e-06,197,268
3,20,0.000018,1.103968e-05,143,192
4,25,0.000007,8.943246e-06,107,150
5,30,0.000020,1.727589e-05,83,124
6,35,0.000103,1.923458e-04,72,104
7,40,0.000028,2.987867e-05,60,86
8,45,0.000012,2.040909e-05,49,76
9,50,0.000124,3.006211e-04,42,66


In [88]:
df[['ttest_pval', 'mw_pval', 'epoch_length']].plot(x='epoch_length')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='epoch_length'>

In [89]:
df[['n_epochs_bl', 'n_epochs_rec', 'epoch_length']].plot(x='epoch_length')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='epoch_length'>